In [1]:
import pandas as pd
import json
from tqdm.autonotebook import tqdm

/Users/jarana/anaconda3/envs/torch1/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


In [1217]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")

In [1847]:
df_action = pd.read_csv("data/AKG/Test Collection/AM/akg_standard_am_verb_object_rele.csv")

In [35]:
import json

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})

In [1876]:
import json

with open("data/AKG/Formal Run Topics/AM_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, entityurl, entity, entityType = [], [], [], []
    for p in data['queries'][0]:
        qid.append(p['queryId'])
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        entityurl.append(p['entityurl'])
am_topic = pd.DataFrame({"query_id": qid, "url": entityurl, "entity": entity, "entityType": entityType})

In [156]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value
with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']

qids = []
props = []
for qid in run['1']:
    tmp = list(run['1'][str(qid)].keys())
    qids.extend([int(qid)] * len(tmp))
    props.extend(tmp)
df_run = pd.DataFrame({"query_id": qids, "property": props})


In [2395]:
df_run = df_run.merge(topic, how="left", on="query_id")

In [1223]:
df = df.merge(topic, how="inner", on="query_id")
df['query'] = df['query'].str.lower()
df['action'] = df['action'].str.lower()
df['entityType'] = df['entityType'].str.lower()
df['entity'] = df['entity'].str.lower()

df['action'] = [preprocessingText(i) for i in df['action']]

In [367]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)
    

In [368]:
selected_qids = df[df['query'].isin(list(entity2wiki.keys()))].query_id.unique()

In [1843]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

res = []
for i in qrel:
    
    if int(i) not in selected_qids:
        continue
    # dict of { document ID: relevance level }
    qrels = qrel[str(i)] 
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
#     grades = list(set(qrels.values())) + [0]
    ranked_list = list(run['1'][str(i)].keys()) # a list of document IDs
#     ranked_list = naive[str(i)] # a list of document IDs
#     ranked_list =  {**naive[str(i)],  **run['1'][str(i)]}
#     for k in ranked_list:
#         ranked_list[k] *= -1

    # labeling: [doc_id] -> [(doc_id, rel_level)]
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)

    # compute the number of documents for each relevance level
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)

    # Let's compute nDCG@5
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    res.append(result)

print(np.mean(res))

0.5196837307322266


In [1838]:
a = sorted(qrels.items(), key=lambda x: x[1], reverse=True)

In [26]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to", "")

In [56]:
df_wiki[df_wiki.headline.str.contains("pizza")].title.tolist()

[' make homeade pizza treats for your hamster',
 ' make homeade pizza treats for your hamster',
 ' plan a spring break staycation1',
 ' eat healthy at an airport',
 ' enjoy your last night on a carnival cruise',
 ' be more spontaneous',
 ' make a solar oven out of cola aluminum cans',
 ' make a pizza box solar oven',
 ' make a pizza box solar oven',
 ' ask a girl to prom4',
 ' ski',
 " deliver a domino's pizza",
 ' say things in italian',
 ' communicate with a non native english speaker',
 ' teach food fractions for first grade math1',
 ' teach food fractions for first grade math1',
 ' teach food fractions for first grade math2',
 ' teach food fractions for first grade math2',
 ' teach food fractions for first grade math2',
 ' figure cost per square inch of pizza1',
 ' figure cost per square inch of pizza1',
 ' figure cost per square inch of pizza1',
 ' figure cost per square inch of pizza2',
 ' figure cost per square inch of pizza2',
 ' figure cost per square inch of pizza2',
 ' figur

In [59]:
context = " ".join(df_wiki[df_wiki.title == ' order a pizza over the phone1'].text.tolist())

In [61]:
context = context.replace("\n","")

In [62]:
context

' make sure you order from a pizza restaurant that accepts orders over the phone for delivery or pickup/takeout. check the website for the pizza place, or call their phone number to briefly ask if they do deliveries or will let you order something in advance and pick it up when it’s ready.if you want the pizza delivered, check to make sure that you live within an area that the pizza place will deliver to. if you’re outside the city limits of where the restaurant is, for instance, that may be beyond their maximum mileage for deliveries.make sure you also have a type of payment that the pizza restaurant accepts. for instance, you will likely want to have cash for a delivery and not plan to use a check in stores that don’t accept them. have several types of payments ready just in case.;  choose whether you’d like to have your pizza delivered straight to your door, or simply go pick it up from the store location. deliveries are great for convenience, while carry-outs are good when you’re o

In [51]:
df_wiki['overview'] = df_wiki['overview'].astype(str)
df_wiki['headline'] = df_wiki['headline'].astype(str)
df_wiki['text'] = df_wiki['text'].astype(str)
df_wiki['sectionLabel'] = df_wiki['sectionLabel'].astype(str)
df_wiki['title'] = df_wiki['title'].astype(str)

In [52]:
df_wiki['overview'] = df_wiki['overview'].str.lower()
df_wiki['title'] = df_wiki['title'].str.lower()
df_wiki['text'] = df_wiki['text'].str.lower()
df_wiki['headline'] = df_wiki['headline'].str.lower()
df_wiki['sectionLabel'] = df_wiki['sectionLabel'].str.lower()

In [2065]:
import re

regex = re.compile('[^a-zA-Z0-9]')
#First parameter is the replacement, second parameter is your input string
def preprocessingText(doc):
    doc = regex.sub(' ', doc)
    doc = " ".join([w for w in doc.split() if not w in stop_words])
    return doc

In [1680]:
df_wiki['overview'] = [preprocessingText(i) for i in df_wiki['overview']]
df_wiki['headline'] = [preprocessingText(i) for i in df_wiki['headline']]
df_wiki['text'] = [preprocessingText(i) for i in df_wiki['text']]
df_wiki['sectionLabel'] = [preprocessingText(i) for i in df_wiki['sectionLabel']]
df_wiki['title'] = [preprocessingText(i) for i in df_wiki['title']]

In [340]:
%%time
queries = [q.lower() for q in df['query'].unique()]
query2wiki = collections.defaultdict(object)
for entity in tqdm(queries, total=len(queries)):
    df_ = df_wiki[df_wiki.title.str.contains(entity, na=False)]
    if len(df_) > 0:
        query2wiki[entity] = df_


CPU times: user 2min 11s, sys: 668 ms, total: 2min 12s
Wall time: 2min 12s


In [682]:
query2id = dict(zip(df['query'], df['query_id']))
query2entity = dict(zip(df['query'], df['entity']))
query2action = dict(zip(df['query'], df['action']))

In [456]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jarana/anacon

In [470]:
all_properties = []
for i in run["1"]:
    all_properties.extend(list(run["1"][i].keys()))

In [1231]:
matched_wiki = df_wiki[df_wiki.text.str.contains('|'.join(queries), na=False)]
wiki_all_text = []
for col in df_wiki.columns:
    wiki_all_text.extend(matched_wiki[col].unique())

In [1704]:
corpus = []
for name, group in groups:
    title = name
    overview = group['overview'].unique().tolist()
    text = group['text'].unique().tolist()
    headline = group['headline'].unique().tolist()
    sectionLabel = group['sectionLabel'].unique().tolist()
#     print(overview, text, headline, sectionLabel, title)
    doc = " ".join([title] + overview + sectionLabel + headline + text)
    corpus.append(doc)

In [2111]:
groups = df_wiki.groupby("title")

In [2115]:
corpus = []
for name, group in groups:
    print(gro)
#     corpus.append(preprocessingText(" ".join([name] + group.text.tolist())))

In [1486]:
def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return [m.group(0) for m in matches]

In [2116]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [2131]:
qidMS = []
docMS = []
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    query = row['action'] + " " + row['entity'] + " " + row['entityType']
    query = query.replace("thing", "")
    tokenized_query = query.split(" ")
    docs = bm25.get_top_n(tokenized_query, corpus, n=5)
    qidMS.append(row['query_id'])
    docMS.append([preprocessingText(doc) for doc in docs])


In [2137]:
docMS2vec = {}
for d, i in zip(docMS, qidMS):
    data = getVectors(d)
    docMS2vec[i] = [data[k] for k in data]


In [2180]:
entity2vec = {}
for i, j, k, l in df[["query_id", "entity", "action", "entityType"]].drop_duplicates().values:
    q = k +" " +j + " " + l
    q = q.replace("thing", "")
    data = getVectors([q])
    entity2vec[i] = data[q]

In [1780]:
cand_props = []
for i in df['query_id'].unique():
    cand_props.extend(run['1'][str(i)].keys())
cand_props = list(set(cand_props))


In [1781]:
propMS2vec = {}
for i in range(0, len(cand_props), 20):
    data = getVectors(cand_props[i:i+20])
    for i in data:
        propMS2vec[i] = data[i]

In [2015]:
run_props = []
for i in run['1']:
    run_props.extend(list(run['1'][i].keys()))
    

In [2016]:
run_props = set(run_props)

In [2472]:
unseen_prop = list(set.difference(a, run_props))

In [2476]:
df[df.property == "offer"]

,query_id,property,rele_label,query,entity,entityType,action


# MS Turing

In [2421]:
df_run['entityType'] = df_run['entityType'].str.lower()
type2prop = df_run.groupby("entityType")['property'].unique().to_dict()

In [2449]:
df_train = df_run[["query_id", "property"]].merge(df[df.query_id.isin(trainIds)], how="right", on=["query_id", "property"])

In [2470]:
res = []
res2 = []
# goodType = set([])
for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
    if row['query_id'] not in testIds:
        continue
    qrels = qrel[str(row['query_id'])]
    cand_properties = type2prop[row['entityType']]
    
    rank = {}
    for p in cand_properties:

#         score = cosine_similarity([entity2vec[row['query_id']]], [propMS2vec[p]])[0][0]
#         score = [cosine_similarity([i], [propMS2vec[p]])[0][0] for i in docMS2vec[row['query_id']]]
#         rank[p] = score
        if p not in row['entityType']:
            rank[p] = 0
        else:
            rank[p] = popProp[row['entityType']][p]

    rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
    our = evaluate(qrels, rank)
#     our = evaluate(qrels, cand_properties)
    base = evaluate(qrels, list(run["1"][str(row["query_id"])].keys()))

    res.append(our)
    res2.append(base)
print(np.mean(res))
print(np.mean(res2))

0.5238514072501347
0.5347861565889339


In [2362]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate, Multiply

class PairwiseModel():
    def __init__(self):
        q_inp = Input(shape=(100,))
        t_inp = Input(shape=(100,))
# #         embedding_layer = Dense(100, activation="linear")
#         q_emb = embedding_layer(q_inp)
#         t_emb = embedding_layer(t_inp)
    #     q_emb = lstm(embedding_layer(q_inp))
    #     t_emb = lstm(embedding_layer(t_inp))
        pred = Multiply()([q_inp, t_inp])
#         pred = Multiply()([q_emb, t_emb])
        dense = Dense(1, activation="sigmoid")
        pred = dense(pred)
        self.model = Model([q_inp, t_inp], pred)

        self.model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])

m = PairwiseModel()

In [2373]:
from keras.layers import Input, Embedding, Dot, Subtract, Activation, SimpleRNN, Flatten, Lambda
from keras.models import Model
from keras import backend as K
import numpy as np

def bpr_triplet_loss(X):
    positive_item_latent, negative_item_latent = X

    loss = 1 - K.log(K.sigmoid(positive_item_latent - negative_item_latent))

    return loss


def identity_loss(y_true, y_pred):
    return K.mean(y_pred - 0 * y_true)


class BPR():
    def __init__(self):

        self.queryInput = Input(shape=(100,))
        self.propPosInput = Input(shape=(100,))
        self.propNegInput = Input(shape=(100,))

        queryEmbeddingLayer = Dense(100, name="uEmb")
        propEmbeddingLayer = Dense(100, name="iEmb")

        self.qEmb = queryEmbeddingLayer(self.queryInput)
        self.pEmb = propEmbeddingLayer(self.propPosInput)
        self.nEmb = propEmbeddingLayer(self.propNegInput)

        pDot = Dot(axes=-1)([self.qEmb, self.pEmb])
        nDot = Dot(axes=-1)([self.qEmb, self.nEmb])
        #
        # diff = Subtract()([pDot, nDot])
        #
        lammbda_output = Lambda(bpr_triplet_loss, output_shape=(1,))
        self.pred = lammbda_output([pDot, nDot])

        self.model = Model(inputs=[self.queryInput, self.propPosInput, self.propNegInput], outputs=self.pred)

        self.model.compile(optimizer="adam", loss=identity_loss)
        self.predictor = Model([self.queryInput, self.propPosInput], [pDot])
model = BPR()

In [2363]:
x_query = [entity2vec[i] for i in df[df.query_id.isin(trainIds)].query_id]
x_prop = [propMS2vec[i] if i in propMS2vec else np.zeros(100) for i in df[df.query_id.isin(trainIds)].property]
y = df[df.query_id.isin(trainIds)].rele_label.values

In [2367]:
for z in range(20):
    m.model.fit([x_query, x_prop], y, verbose=0)
    check(m)
    

0.4584756296210826
0.44866069150007026
111
0.4601898985848593
0.44866069150007026
111
0.4597240833255444
0.44866069150007026
111
0.4584485044563727
0.44866069150007026
111
0.4592697099296743
0.44866069150007026
111
0.4592126712938601
0.44866069150007026
111
0.45916535668859143
0.44866069150007026
111
0.4579256456394781
0.44866069150007026
111
0.45839349964250375
0.44866069150007026
111
0.45925031941791855
0.44866069150007026
111
0.4597902454216229
0.44866069150007026
111
0.46077515342551656
0.44866069150007026
111
0.46052600609332545
0.44866069150007026
111
0.4604458214490861
0.44866069150007026
111
0.46043371923957005
0.44866069150007026
111
0.46049690008623656
0.44866069150007026
111
0.4606618436776602
0.44866069150007026
111
0.461124507647858
0.44866069150007026
111
0.4624650811793885
0.44866069150007026
111
0.4622547890272381
0.44866069150007026
111


In [2416]:
def check(m):
    res = []
    res2 = []
    # goodType = set([])
    for idx, row in df[['query_id', 'entity', 'action', 'entityType']].drop_duplicates().iterrows():
        if row['query_id'] not in testIds:
            continue

        qrels = qrel[str(row['query_id'])]
        cand_properties = list(runPropType[row['entityType']].keys())


        if len(cand_properties) == 0:
            continue
        rank = {}
        for p in cand_properties:
            if p in unseen_prop:
                continue

    #         score = cosine_similarity(docMS2vec[row['query_id']], [propMS2vec[p]])[0][0]
    #         score = [cosine_similarity([i], [propMS2vec[p]])[0][0] for i in docMS2vec[row['query_id']]]
            rank[p] = m.model.predict([[entity2vec[row['query_id']]], [propMS2vec[p]]])[0][0]

        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        our = evaluate(qrels, rank)
        base = evaluate(qrels, cand_properties)

        res.append(our)
        res2.append(base)
    print(np.mean(res))
    print(np.mean(res2))

In [2455]:
from sklearn.model_selection import train_test_split

popProp = collections.defaultdict(dict)
runPropType = collections.defaultdict(dict)
trainIds, testIds = [], []
for name, group in df_train.groupby("entityType"):
    ids = group.query_id.unique().tolist()
    if len(ids) < 2:
        trainIds.extend(ids)
        continue
    train, test = train_test_split(np.array(ids), train_size=0.5, random_state=42)
    trainIds.extend(train)
    testIds.extend(test)
#     kf = KFold(n_splits=2)
#     kf.get_n_splits(ids)
#     print(kf.split(ids))
#     for train_index, test_index in kf.split(ids):
#         print(train_index)
#         print(test_index)
#         print(ids)
#         print(ids[[train_index]], ids[[test_index]])
    for idx, row in group.iterrows():
        if row['query_id'] in test:
            continue
        
        if row['property'] not in popProp[name]:
            popProp[name][row['property']] = 0
        popProp[name][row['property']] += int(row['rele_label'])
#         popProp[name][row['property']] = 1



In [2357]:
# from sklearn.model_selection import KFold

# print(kf)
#     kf = KFold(n_splits=2)
#     kf.get_n_splits(X)
# for train_index, test_index in kf.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]

In [505]:
topic_all_text = []
for col in ["property", "query", "entity", "entityType", "action"]:
    topic_all_text.extend(df[col].unique())

In [1197]:
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
raw_text = [str(i) for i in wiki_all_text + topic_all_text + all_properties]
# corpus = [w for w in " ".join(corpus).split() if not w in stop_words]

In [1721]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
import json

def getVectors(queries):
    headers = {
        # Request headers
        'Content-Type': 'application/json',
        'Ocp-Apim-Subscription-Key': '924c1505854b4da4a6144a1cce92937f',
    }

    params = urllib.parse.urlencode({})
    
    try:
        conn = http.client.HTTPSConnection('api.msturing.org')
#         conn.request("POST", "/gen/encode?%s" % params, '{"queries": ["how to make gingerbread people (in grams)", "test AI"]}', headers)
        conn.request("POST", "/gen/encode?%s" % params, str({"queries": queries}).replace("\'", "\""), headers)
        response = conn.getresponse()
        data = response.read()
        data = json.loads(data)
        conn.close()
    except Exception as e:
#         print(data)
        print(e)
#         print("[Errno {0}] {1}".format(e.errno, e.strerror))
    
    return {data[i]['query']:data[i]['vector'] for i in range(len(data))}

In [1186]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([data["startTime"]], [data["time"]])

KeyError: 'time'

In [2479]:
def get_pretrain_embeddings(path, word_index, EMBEDDING_DIM=300):
    MAX_NUM_WORDS = len(word_index)
    BASE_DIR = path + 'data/'
    GLOVE_DIR = os.path.join(BASE_DIR, 'w2v')
    print('Indexing word vectors.')

    embeddings_index = {}
    with open(os.path.join(GLOVE_DIR, 'glove.6B.50d.txt'), encoding="utf-8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, 'f', sep=' ')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))

    print('Preparing embedding matrix.')

    # prepare embedding matrix
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    found = 0
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if embedding_vector.shape[0] == 0:
                continue
            embedding_matrix[i] = embedding_vector
            found += 1

    print("Token num: %d, Found Tokens: %d" % (len(word_index), found))

    # load pre-trained word embeddings into an Embedding layer
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix))

    return embedding_layer

Found 218841 unique tokens.


In [1349]:
from Models import getDSSM, getRanker

path = "/Users/jarana/workspace/WikiHow-Task-Based/"
max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)

Indexing word vectors.
Found 1917494 word vectors.
Preparing embedding matrix.
Token num: 101910, Found Tokens: 69394


In [1789]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    t_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
#     lstm = LSTM(100)
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    t_emb = GlobalMaxPooling1D()(embedding_layer(t_inp))
#     q_emb = lstm(embedding_layer(q_inp))
#     t_emb = lstm(embedding_layer(t_inp))
    pred = Dot(-1, normalize=True)([q_emb, t_emb])
    model = Model([q_inp, t_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
MAX_SEQUENCE_LENGTH = int(np.mean([len(i.split()) for i in corpus]))
model = getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH)

In [ ]:
x_task = pad_sequences(tokenizer.texts_to_sequences(data["t"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query1 = pad_sequences(tokenizer.texts_to_sequences(data["q1"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query2 = pad_sequences(tokenizer.texts_to_sequences(data["q2"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
y = np.array([[i,j] for i,j in zip(data.label1.values, data.label2.values)])

In [1912]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    return result

# Glove

In [1793]:
res = []
for idx, row in df[['query_id', 'entity', 'action']].drop_duplicates().iterrows():
    qrels = qrel[str(row['query_id'])]
    cand_properties = list(run['1'][str(row['query_id'])].keys())
    
    
    query = row['action'] + " " +row['entity']
    tokenized_query = query.split(" ")
    doc = bm25.get_top_n(tokenized_query, corpus, n=1)[0]
#     print(qrels)
#     print(cand_properties)
#     cand_properties = list(run['1'][str(query2id[query])].keys())
#     wikis = [str(i) for i in query2wiki[query][col].unique()]
    x_wikis = pad_sequences(tokenizer.texts_to_sequences([doc]), maxlen=MAX_SEQUENCE_LENGTH)
#     print(x_wikis)
    rank = {}
    for prop in cand_properties:
#         x_props = pad_sequences(tokenizer.texts_to_sequences([row['entity']+ " " + row['action'] + " " + prop]), maxlen=MAX_SEQUENCE_LENGTH)
        x_props = pad_sequences(tokenizer.texts_to_sequences([prop]), maxlen=MAX_SEQUENCE_LENGTH)
#         x_props = pad_sequences(tokenizer.texts_to_sequences([prop]), maxlen=MAX_SEQUENCE_LENGTH)
        rank[prop] = model.predict([x_props, x_wikis])[0][0]
    rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
    
    res.append(evaluate(qrels, rank))
print(np.mean(res))

0.5076663049135386


In [1423]:
trainIds, testIds = [], []
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() > 1:
        ids = list(group.query_id.unique())
        mid = int(group.query_id.nunique() / 2)
        trainIds.extend(ids[:mid])
        testIds.extend(ids[mid:])
    else:
        ids = list(group.query_id.unique())
        trainIds.extend(ids)

In [1497]:
MAX_PROP_SEQUENCE_LENGTH = max([len(i.split()) for i in props])

In [1498]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH, MAX_PROP_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_QUERY_SEQUENCE_LENGTH,))
    d_inp = Input(shape=(MAX_DOC_SEQUENCE_LENGTH,))
    p_inp = Input(shape=(MAX_PROP_SEQUENCE_LENGTH,))
    
    
    q_lstm = LSTM(100)
    d_lstm = LSTM(100)
#     q_emb = q_lstm(embedding_layer(q_inp))
#     d_emb = d_lstm(embedding_layer(d_inp))
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    d_emb = GlobalMaxPooling1D()(embedding_layer(d_inp))
    p_emb = GlobalMaxPooling1D()(embedding_layer(p_inp))
    
    concat = Concatenate()([q_emb, d_emb, p_emb])
    dense = Dense(1, activation='sigmoid')
    pred = dense(concat)
    model = Model([q_inp, d_inp, p_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [1425]:
from keras.utils import to_categorical

In [1426]:
prop2id = {j:i for i, j in zip(range(df.property.nunique()), df.property.unique())}
id2prop = {i:j for i, j in zip(range(df.property.nunique()), df.property.unique())}

In [1427]:
PROPERTY_NUN = df.property.nunique() 


In [1428]:
MAX_DOC_SEQUENCE_LENGTH = int(np.mean([len(i.split()) for i in corpus]))

In [1657]:
def get_train_data():
    train_id, train_query, train_prop, train_doc, train_y = [], [], [], [], []
    test_id, test_query, test_prop, test_doc, test_y = [], [], [], [], []

    for idx, row in tqdm(df[['query_id', 'entity', 'action']].drop_duplicates().iterrows()):

    #     qrels = qrel[str(row['query_id'])]
    #       convert prop to prop_id
    #     label = to_categorical([prop2id[i] for i in list(qrels.keys())], num_classes=PROPERTY_NUN)
    #     label = np.sum(label, axis=0)
        label = 1

        query = row['action'] + " " +row['entity']
        tokenized_query = query.split(" ")
        doc = bm25.get_top_n(tokenized_query, corpus, n=1)[0]

        x_query = pad_sequences(tokenizer.texts_to_sequences([query]), maxlen=MAX_QUERY_SEQUENCE_LENGTH)[0]
        doc = pad_sequences(tokenizer.texts_to_sequences([doc]), maxlen=MAX_DOC_SEQUENCE_LENGTH)[0]

        pos_props = [" ".join(camel_case_split(i)) for i in df[df.query_id == row['query_id']].property.tolist()]
        for pp in pos_props:
            label = [1,0]
            prop = pad_sequences(tokenizer.texts_to_sequences([pp]), maxlen=MAX_PROP_SEQUENCE_LENGTH)[0]

            train_prop.append(prop) if row['query_id'] in trainIds else test_prop.append(prop)

            neg_prop = random.sample(df.property.unique().tolist(), k=1)
            while neg_prop in pos_props:
                neg_prop = random.sample(df.property.unique().tolist(), k=1)
            neg_prop = pad_sequences(tokenizer.texts_to_sequences([neg_prop]), maxlen=MAX_PROP_SEQUENCE_LENGTH)[0]
            train_prop.append(neg_prop) if row['query_id'] in trainIds else test_prop.append(neg_prop)

            for i in range(2):
                train_query.append(x_query) if row['query_id'] in trainIds else test_query.append(x_query)
                train_doc.append(doc) if row['query_id'] in trainIds else test_doc.append(doc)
                train_id.append(row['query_id']) if row['query_id'] in trainIds else test_id.append(row['query_id'])

            train_y.extend(label) if row['query_id'] in trainIds else test_y.extend(label)
    #         rank = {}
    #         for prop in cand_properties:
    #             x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop] * len(x_wikis)), maxlen=MAX_SEQUENCE_LENGTH)
    #             rank[prop] = np.max(model.predict([x_props, x_wikis]))
    #         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
    #         res.append(evaluate(qrels, rank))
    train_query = np.array(train_query)
    train_doc = np.array(train_doc)
    train_prop = np.array(train_prop)
    train_y = np.array(train_y)
    test_query = np.array(test_query)
    test_doc = np.array(test_doc)
    test_prop = np.array(test_prop)
    test_y = np.array(test_y)
    
    return train_query, train_doc, train_prop, train_y, train_id, test_query, test_doc, test_prop, test_y, test_id


In [1658]:
model = getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH, MAX_PROP_SEQUENCE_LENGTH)


In [1659]:
def eval_model(model):
    pred = model.predict([test_query, test_doc])  
    sort_pred = np.argsort(pred)[:,::-1][:,:20]
    runLSTM = {}
    for i, j in zip(test_id, sort_pred):
        runLSTM[i] = [id2prop[k] for k in j]
    for col in df_wiki.columns:
        res = []
        for qid in testIds:
            qrels = qrel[str(qid)]
            cand_properties = runLSTM[qid]
            res.append(evaluate(qrels, cand_properties))
        print(col, np.mean(res))    
        break
    

for i in range(5):
    train_query, train_doc, train_prop, train_y, train_id, test_query, test_doc, test_prop, test_y, test_id = get_train_data()
    model.fit([train_query, train_doc, train_prop], train_y, validation_data=([test_query, test_doc, test_prop], test_y), verbose=2)
    eval_pair_model(model)



Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 47s - loss: 0.3834 - acc: 0.8118 - val_loss: 0.3175 - val_acc: 0.8253


hit and run ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
language acquisition ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
mediation [

knowing ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
krrish ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
law & order: special victims 

history of television ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
insomnia ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
pubis ['speci

miami heat ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
motorola ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'isAccessoryOrSparePartFor', 'vehicleModelDate', 'vehicleInteriorType', 'driveWheelConfiguration', 'productID', 'mainEntityOfPage', 'amenityFeature', 'vehicleIdentificationNumber', 'numberOfForwardGears', 'taxID', 'vehicleSeatingCapacity', 'previousStartDate', 'foundingDate', 'broadcastServiceTier']
nvidia ['specialOpeningH

...


Train on 3878 samples, validate on 3714 samples
Epoch 1/1
 - 43s - loss: 0.3028 - acc: 0.8515 - val_loss: 0.2823 - val_acc: 0.8479


hit and run ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'vehicleModelDate', 'isAccessoryOrSparePartFor', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'vehicleInteriorType', 'driveWheelConfiguration', 'vehicleSeatingCapacity', 'productID', 'amenityFeature', 'vehicleIdentificationNumber', 'relatedCondition', 'mainEntityOfPage', 'vehicleInteriorColor', 'taxID', 'numberOfForwardGears', 'additionalType']
language acquisition ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'vehicleModelDate', 'isAccessoryOrSparePartFor', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'vehicleInteriorType', 'driveWheelConfiguration', 'vehicleSeatingCapacity', 'productID', 'amenityFeature', 'vehicleIdentificationNumber', 'relatedCondition', 'mainEntityOfPage', 'vehicleInteriorColor', 'taxID', 'numberOfForwardGears', 'additionalType']
mediation

knowing ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'vehicleModelDate', 'isAccessoryOrSparePartFor', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'vehicleInteriorType', 'driveWheelConfiguration', 'vehicleSeatingCapacity', 'productID', 'amenityFeature', 'vehicleIdentificationNumber', 'relatedCondition', 'mainEntityOfPage', 'vehicleInteriorColor', 'taxID', 'numberOfForwardGears', 'additionalType']
krrish ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'vehicleModelDate', 'isAccessoryOrSparePartFor', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'vehicleInteriorType', 'driveWheelConfiguration', 'vehicleSeatingCapacity', 'productID', 'amenityFeature', 'vehicleIdentificationNumber', 'relatedCondition', 'mainEntityOfPage', 'vehicleInteriorColor', 'taxID', 'numberOfForwardGears', 'additionalType']
law & order: special victim

history of television ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'vehicleModelDate', 'isAccessoryOrSparePartFor', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'vehicleInteriorType', 'driveWheelConfiguration', 'vehicleSeatingCapacity', 'productID', 'amenityFeature', 'vehicleIdentificationNumber', 'relatedCondition', 'mainEntityOfPage', 'vehicleInteriorColor', 'taxID', 'numberOfForwardGears', 'additionalType']
insomnia ['specialOpeningHoursSpecification', 'dateVehicleFirstRegistered', 'broadcastChannelId', 'openingHoursSpecification', 'vehicleModelDate', 'isAccessoryOrSparePartFor', 'globalLocationNumber', 'feesAndCommissionsSpecification', 'vehicleInteriorType', 'driveWheelConfiguration', 'vehicleSeatingCapacity', 'productID', 'amenityFeature', 'vehicleIdentificationNumber', 'relatedCondition', 'mainEntityOfPage', 'vehicleInteriorColor', 'taxID', 'numberOfForwardGears', 'additionalType']
pubis ['spe

KeyboardInterrupt: 

In [1831]:
# res = []
for qid in testIds:
    qrels = qrel[str(qid)]
    cand_properties = list(run['1'][str(qid)].keys())
    res.append(evaluate(qrels, cand_properties))
print(np.mean(res))    
    



0.5149226462923876


In [1612]:
    pred = model.predict([test_query, test_doc, test_prop])  


In [1621]:
        query = row['action'] + " " +row['entity']
        tokenized_query = query.split(" ")
        doc = bm25.get_top_n(tokenized_query, corpus, n=1)[0]

        x_query = pad_sequences(tokenizer.texts_to_sequences([query]), maxlen=MAX_QUERY_SEQUENCE_LENGTH)[0]
        x+doc = pad_sequences(tokenizer.texts_to_sequences([doc]), maxlen=MAX_DOC_SEQUENCE_LENGTH)[0]


array([[    0,     0,     0,     0,   635,   695],
       [    0,     0,     0,     0,  5510,  1180],
       [    0,     0,     0,     0,     0,  9792],
       ...,
       [    0,     0,     0,     0,     0,   208],
       [    0,     0,     0,     0,     0, 14356],
       [    0,     0,     0,     0, 88884,   688]], dtype=int32)

In [1645]:
prop2realprop = {" ".join(camel_case_split(i)):i for i in df.property.unique().tolist()}

In [1666]:
def eval_pair_model(model):
    all_props = df.property.unique().tolist()
    res = []
    for idx, row in tqdm(df[['query_id', 'entity', 'action']].drop_duplicates().iterrows()):
        if row['query_id'] not in testIds:
            continue
            
        query = row['action'] + " " +row['entity']
        tokenized_query = query.split(" ")
        doc = bm25.get_top_n(tokenized_query, corpus, n=1)[0]
        
#         _props = list(run['1'][str(row['query_id'])].keys())
        _props = props


        x_query = pad_sequences(tokenizer.texts_to_sequences([query] * len(list(set(_props)))), maxlen=MAX_QUERY_SEQUENCE_LENGTH)
        x_doc = pad_sequences(tokenizer.texts_to_sequences([doc] * len(list(set(_props)))), maxlen=MAX_DOC_SEQUENCE_LENGTH)
        
        cand_properties = list(run['1'][str(row['query_id'])].keys())

        x_prop = pad_sequences(tokenizer.texts_to_sequences(list(set(_props))), maxlen=MAX_PROP_SEQUENCE_LENGTH)
        
        pred = model.predict([x_query, x_doc, x_prop])  
        rank = {}
        for i, j in zip(list(set(props)), pred.flatten()):
            rank[prop2realprop[i]] = j
            
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)][:20]
        qrels = qrel[str(row['query_id'])]
#                 cand_properties = runLSTM[qid]
#         print(row['entity'], rank)
        res.append(evaluate(qrels, rank))
    print(np.mean(res))
eval_pair_model(model)


0.03585620819659419


In [1794]:
len(df)

3796

In [1796]:
df.action.unique()

array(['have', 'take', 'brainstorm', 'start', 'get funding',
       'bleach hair back to brown', 'stop at the scene of an accident',
       'suggest a foreign language thats easy to learn',
       'help you come up with possible solutions',
       'do any aerobic physical activity', 'know who do the spoof ',
       'use on fabric', 'know japanese recipe ingredient translation',
       'forget', 'remember', 'love that movie', 'give fives', 'follow',
       'leave', 'happen', 'prepare for a shakespeare play', 'choose',
       'abandon', 'obsess', 'come', 'watch the devil wears prada',
       'watch full episode of robot chicken',
       'watch movie with english subtitle', 'make love with a girl',
       'reclaim', 'see the spirits of people who have died',
       'meet a good woman', 'lose', 'make trust',
       'watch porco rosso ( 1992 )', 'love the colors of prom',
       'find the name of a song ', 'see', 'win emmy', 'release on dvd',
       'watch this video', 'give', 'become a f1 